# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv",index_col = "City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,waitangi,-43.9535,-176.5597,8.79,87,78,2.68,NZ,1717078467
1,jamestown,42.0970,-79.2353,13.77,71,20,4.12,US,1717078478
2,ghat,24.9647,10.1728,37.74,12,0,2.78,LY,1717078517
3,ushuaia,-54.8000,-68.3000,1.81,64,20,6.69,AR,1717078482
4,sao joao da barra,-21.6403,-41.0511,22.05,65,98,7.51,BR,1717078721


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot

map_plot = city_data_df.hvplot.points(
   "Lng",
   "Lat",
    geo = True,
    tiles ="OSM",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 1.0,
    color ="City",
    alpha = 0.5
    )


# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df['Max Temp']>=20 )]
city_data_df = city_data_df.loc[(city_data_df['Cloudiness']==0) ]
city_data_df = city_data_df.loc[(city_data_df['Max Temp'] < 30) ]
city_data_df = city_data_df.loc[(city_data_df['Wind Speed']<4.5)]
                         
# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head(30)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
13,toliara,-23.3500,43.6667,24.84,61,0,3.76,MG,1717078721
25,farafangana,-22.8167,47.8333,22.09,68,0,2.78,MG,1717078722
34,karratha,-20.7377,116.8463,22.65,68,0,0.90,AU,1717078723
45,kailua-kona,19.6406,-155.9956,23.98,82,0,2.57,US,1717078499
93,rondonopolis,-16.4708,-54.6356,26.88,40,0,1.90,BR,1717078729
116,lazaro cardenas,17.9583,-102.2000,27.60,76,0,1.22,MX,1717078731
136,ankazoabo,-22.2833,44.5167,25.50,29,0,0.87,MG,1717078733
137,short pump,37.6504,-77.6125,21.64,65,0,2.57,US,1717078733
142,shakawe,-18.3667,21.8500,27.46,16,0,2.22,BW,1717078733


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
13,toliara,MG,-23.3500,43.6667,61,
25,farafangana,MG,-22.8167,47.8333,68,
34,karratha,AU,-20.7377,116.8463,68,
45,kailua-kona,US,19.6406,-155.9956,82,
93,rondonopolis,BR,-16.4708,-54.6356,40,
116,lazaro cardenas,MX,17.9583,-102.2000,76,
136,ankazoabo,MG,-22.2833,44.5167,29,
137,short pump,US,37.6504,-77.6125,65,
142,shakawe,BW,-18.3667,21.8500,16,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
          "apiKey":geoapify_key
        
          
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"]  = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
toliara - nearest hotel: Ambary
farafangana - nearest hotel: Le Concombre Masqué
karratha - nearest hotel: Karratha International Hotel
kailua-kona - nearest hotel: Kona Seaside Hotel
rondonopolis - nearest hotel: Hotel Sumaré
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
ankazoabo - nearest hotel: No hotel found
short pump - nearest hotel: Hyatt House Richmond / Short Pump
shakawe - nearest hotel: EdenWood guesthouse
giyani - nearest hotel: No hotel found
hammerfest - nearest hotel: Thon Hotel Hammerfest
vadso - nearest hotel: Scandic Vadsø
laguna - nearest hotel: Holiday Inn Express & Suites
mariehamn - nearest hotel: Hotell Esplanad
xai-xai - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
ostersund - nearest hotel: Hotell Stortorget
tsiombe - nearest hotel: No hotel found
mount washington - nearest hotel: No hotel found
leninskoye - nearest hotel: No hotel found
caldas novas - nearest hotel: Hotel Roma
buka - nea

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
13,toliara,MG,-23.3500,43.6667,61,Ambary
25,farafangana,MG,-22.8167,47.8333,68,Le Concombre Masqué
34,karratha,AU,-20.7377,116.8463,68,Karratha International Hotel
45,kailua-kona,US,19.6406,-155.9956,82,Kona Seaside Hotel
93,rondonopolis,BR,-16.4708,-54.6356,40,Hotel Sumaré
116,lazaro cardenas,MX,17.9583,-102.2000,76,Hotel Sol del Pacífico
136,ankazoabo,MG,-22.2833,44.5167,29,No hotel found
137,short pump,US,37.6504,-77.6125,65,Hyatt House Richmond / Short Pump
142,shakawe,BW,-18.3667,21.8500,16,EdenWood guesthouse


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles ="OSM",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 1.5,
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"]
    )

# Display the map 
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)